In [17]:
import subprocess
import pandas as pd
import os
import glob
from tqdm import tqdm
from os.path import basename as bn, join, split as sp
import parselmouth
import numpy as np
from parselmouth.praat import call
from scipy.io.wavfile import write
import praat_formants_python as pfp


# DATASET Path and Constants

In [15]:
# Write paths
ALL_EXP_FOLDER = "./exports/"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(ALL_EXP_FOLDER) #make export folder

# Synth Vowel info Export CSV filename
SYNTH_VOWEL_INFO_IMP_FILENAME = "d_base_formants_synth_vowels_vowlimLIM.csv"
SYNTH_VOWEL_INFO_IMP_FILEPATH = join(ALL_EXP_FOLDER, SYNTH_VOWEL_INFO_IMP_FILENAME)

# Vowel subset Export CSV filename
SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME = "e_synth-vowels_formant_estimation_vowlimLIM.csv"
SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH = join(ALL_EXP_FOLDER, SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME)


### Import Synth Vowel (BASE) Formant Info dataframe

In [16]:
VOWEL_LIMIT = 10
SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF = pd.read_csv(SYNTH_VOWEL_INFO_IMP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}"))
SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF

,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,F2_median_praat_base,F3_median_praat_base,synth_vowel_path
0,0,MTRT0,M,0.105000,iy,97.40,456.21,1906.70,2309.29,454.47,1921.02,2346.71,./audio_exports/vowlim10/iy_0_MTRT0_M_97.wav
1,1,MMGG0,M,0.093812,iy,122.09,561.03,2072.62,2771.38,314.55,2110.85,2724.62,./audio_exports/vowlim10/iy_1_MMGG0_M_122.wav
2,2,MPPC0,M,0.073125,iy,142.78,466.98,1948.16,2673.15,463.02,1958.39,2700.85,./audio_exports/vowlim10/iy_2_MPPC0_M_143.wav
3,3,MKAH0,M,0.061125,iy,126.96,480.70,2161.41,2724.09,481.49,2157.63,2737.17,./audio_exports/vowlim10/iy_3_MKAH0_M_127.wav
4,4,MRML0,M,0.177437,iy,86.21,880.29,2167.21,2948.52,355.54,2210.83,2897.05,./audio_exports/vowlim10/iy_4_MRML0_M_86.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,FCJF0,F,0.040438,ax-h,206.60,1095.70,1784.74,3048.24,1018.81,1767.53,2987.76,./audio_exports/vowlim10/ax-h_395_FCJF0_F_207.wav
396,396,FVMH0,F,0.046250,ax-h,215.07,758.37,2162.47,3188.13,541.80,2153.17,3176.38,./audio_exports/vowlim10/ax-h_396_FVMH0_F_215.wav
397,397,FSMA0,F,0.030000,ax-h,207.91,395.79,1155.12,2881.54,386.55,1252.54,2838.76,./audio_exports/vowlim10/ax-h_397_FSMA0_F_208.wav
398,398,FCAJ0,F,0.016813,ax-h,190.54,511.05,1749.78,2994.59,511.05,1749.78,2994.59,./audio_exports/vowlim10/ax-h_398_FCAJ0_F_191.wav


## FUNCTION: Measure formants of synth vowels | PRAAT FORMANTS

In [20]:
def measure_pitch(audio_path: str) -> float:
    f0min, f0max = [75, 600]
    
    sound = parselmouth.Sound(audio_path) # read the sound
    pitch = call(sound, "To Pitch", 0, f0min, f0max) #create a praat pitch object
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch
    return mean_pitch
    
def measure_formants_pfp(audio_path: str, start_sec: float, end_sec: float) -> dict:
    formants: list = pfp.formants_at_interval(
        audio_path, start_sec, end_sec, maxformant=5500, winlen=0.025, preemph=50
    )

    pitch_mean = measure_pitch(audio_path)
    pitch_mean = np.round(pitch_mean, 2)
    
    formants_mean = formants.mean(axis=0)
    formants_mean = list(formants_mean)[1:]  # skip time
    formants_mean = np.round(formants_mean, 2)  # round

    formants_median = np.median(formants, axis=0)
    formants_median = list(formants_median)[1:]  # skip time
    formants_median = np.round(formants_median, 2) # round


    return {
        "pitch_mean_synthvow_praat": pitch_mean,

        "F1_mean_synthvow_praat": formants_mean[0],
        "F2_mean_synthvow_praat": formants_mean[1],
        "F3_mean_synthvow_praat": formants_mean[2],

        "F1_median_synthvow_praat": formants_median[0],
        "F2_median_synthvow_praat": formants_median[1],
        "F3_median_synthvow_praat": formants_median[2],
    }


## FUNCTION: Measure formants of synth vowels | DEEPFORMANTS

In [21]:
def get_deepformants(audio_file: str, begin: float, end: float):
    proc = subprocess.check_output('/home/jeevan/dev/anaconda3/envs/pytorch/bin/python formants.py ' + audio_file + ' /home/jeevan/Desktop/DeepFormants/test.csv --begin ' +  str(begin) + ' --end ' + str(end), cwd="/home/jeevan/Desktop/DeepFormants/", shell=True)
    df = pd.read_csv("/home/jeevan/Desktop/DeepFormants/test.csv")
    return df.iloc[0,1:].tolist()
   

def measure_formants_df(audio_path: str,  start_sec: float, end_sec: float):
    _formants: list = get_deepformants(audio_path, start_sec, end_sec)
    _formants = np.round(_formants, 2)  # round

    return {
        "F1_mean_synthvow_deepformant": _formants[0],
        "F2_mean_synthvow_deepformant": _formants[1],
        "F3_mean_synthvow_deepformant": _formants[2],
        "F4_mean_synthvow_deepformant": _formants[3],
    }


### FUNCTION TESTS

In [22]:
v, audf = SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF.loc[np.random.randint(0, len(SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF)), ["vowel_name", "synth_vowel_path"]]
dur = 1.7/2
start, end = (dur - 0.5), (dur + 0.5)
audf = os.path.realpath(audf)
print(audf, start, end )

print(measure_formants_pfp(audf, start, end))
print(measure_formants_df(audf, start, end))


/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/audio_exports/vowlim10/ow_163_MDCM0_M_104.wav 0.35 1.35
{'pitch_mean_synthvow_praat': 104.0, 'F1_mean_synthvow_praat': 823.39, 'F2_mean_synthvow_praat': 884.88, 'F3_mean_synthvow_praat': 2577.83, 'F1_median_synthvow_praat': 825.95, 'F2_median_synthvow_praat': 878.57, 'F3_median_synthvow_praat': 2579.27}
{'F1_mean_synthvow_deepformant': 593.49, 'F2_mean_synthvow_deepformant': 1667.45, 'F3_mean_synthvow_deepformant': 2718.5, 'F4_mean_synthvow_deepformant': 3818.4}


## FUNCTION: Synth Vowel PITCH, FORMANT Estimation | Praat, DeepFormants, +

In [23]:
def estimate_synth_vowel_formants(vowel_info):
    audio_file = vowel_info["synth_vowel_path"]
    audio_file = os.path.realpath(audio_file)
    vowel_name = vowel_info["vowel_name"]

    mdur = 1.7/2
    start_sec, end_sec = (mdur - 0.5), (mdur + 0.5)

    formant_estimates_pfp = measure_formants_pfp(audio_file, start_sec, end_sec) # estimator 1
    formant_estimates_df = measure_formants_df(audio_file, start_sec, end_sec) # estimator 2
    # TODO: estimators 3, 4, 5

    new_vowel_info = dict(vowel_info) | formant_estimates_pfp | formant_estimates_df

    return new_vowel_info

syvinfo = SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF.loc[np.random.randint(0, len(SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF))]
estimate_synth_vowel_formants(syvinfo)

{'index': 115,
 'person_id': 'FJXM0',
 'sex': 'F',
 'duration_second': 0.0638125,
 'vowel_name': 'ih',
 'pitch_mean_praat_base': 231.25,
 'F1_mean_praat_base': 533.28,
 'F2_mean_praat_base': 2138.84,
 'F3_mean_praat_base': 3135.49,
 'F1_median_praat_base': 539.43,
 'F2_median_praat_base': 2066.91,
 'F3_median_praat_base': 3145.57,
 'synth_vowel_path': './audio_exports/vowlim10/ih_115_FJXM0_F_231.wav',
 'pitch_mean_synthvow_praat': 231.12,
 'F1_mean_synthvow_praat': 493.8,
 'F2_mean_synthvow_praat': 1001.03,
 'F3_mean_synthvow_praat': 2102.35,
 'F1_median_synthvow_praat': 493.96,
 'F2_median_synthvow_praat': 1004.89,
 'F3_median_synthvow_praat': 2101.86,
 'F1_mean_synthvow_deepformant': 483.32,
 'F2_mean_synthvow_deepformant': 1845.76,
 'F3_mean_synthvow_deepformant': 2767.67,
 'F4_mean_synthvow_deepformant': 4003.15}

### Create and export Synth Vowel Formant Estimation Result dataframe

In [26]:
SYNTH_VOWELS_FORMANT_DF = pd.DataFrame([estimate_synth_vowel_formants(syv_i) for i, syv_i in tqdm(SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF[0:5].iterrows())])

csv_path = SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}")
print(csv_path)
if not os.path.exists(csv_path):
    SYNTH_VOWELS_FORMANT_DF.to_csv(csv_path, index=False)

SYNTH_VOWELS_FORMANT_DF

5it [00:06,  1.35s/it]

./exports/e_synth-vowels_formant_estimation_vowlim10.csv


,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,...,F1_mean_synthvow_praat,F2_mean_synthvow_praat,F3_mean_synthvow_praat,F1_median_synthvow_praat,F2_median_synthvow_praat,F3_median_synthvow_praat,F1_mean_synthvow_deepformant,F2_mean_synthvow_deepformant,F3_mean_synthvow_deepformant,F4_mean_synthvow_deepformant
0,0,MTRT0,M,0.105000,iy,97.40,456.21,1906.70,2309.29,454.47,...,458.15,826.35,1919.22,460.99,838.44,1908.68,477.39,1725.66,2265.43,3735.31
1,1,MMGG0,M,0.093812,iy,122.09,561.03,2072.62,2771.38,314.55,...,557.77,825.71,2067.31,565.55,844.14,2067.35,509.84,1883.66,2685.41,4045.47
2,2,MPPC0,M,0.073125,iy,142.78,466.98,1948.16,2673.15,463.02,...,439.34,1060.92,1954.17,439.35,1056.90,1956.03,447.30,1754.09,2545.94,3941.59
3,3,MKAH0,M,0.061125,iy,126.96,480.70,2161.41,2724.09,481.49,...,480.80,841.33,2153.55,483.46,854.01,2152.95,468.94,1846.23,2596.85,3941.90
4,4,MRML0,M,0.177437,iy,86.21,880.29,2167.21,2948.52,355.54,...,859.38,930.87,2156.36,868.41,923.04,2156.27,718.07,2011.46,2886.03,4062.82


# TEST